In [1]:
import pickle

import pandas as pd
from lightgbm import LGBMClassifier

import ipywidgets as widgets
from IPython.display import display

from feature_engine.creation import CyclicalFeatures

In [2]:
## load saved data and metadata
su_med_data = pd.read_csv("./su_prescribed_meds.csv")

## load trained model
with open("./saved_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

In [3]:
unique_su = sorted(su_med_data["su_number"].unique().tolist())

In [4]:
bf_intervals = pd.date_range(start="07:00:00", end="12:00:00", freq="15T")
ln_intervals = pd.date_range(start="12:15:00", end="16:00:00", freq="15T")
em_intervals = pd.date_range(start="16:15:00", end="20:00:00", freq="15T")
bt_intervals = pd.date_range(start="20:15:00", end="23:45:00", freq="15T")
ot_intervals = pd.date_range(start="00:00:00", end="07:00:00", freq="15T")

bf_df = pd.DataFrame(data=bf_intervals, columns=["datetime"])
ln_df = pd.DataFrame(data=ln_intervals, columns=["datetime"])
em_df = pd.DataFrame(data=em_intervals, columns=["datetime"])
bt_df = pd.DataFrame(data=bt_intervals, columns=["datetime"])
ot_df = pd.DataFrame(data=ot_intervals, columns=["datetime"])

In [5]:
def make_time_cyclical(df, dt_column="datetime"):
    
    cyclic_feats = ["Given hour", "Given min", "Week Day", "Month", "Quarter"]
    
    df["Given hour"] = df[dt_column].apply(lambda x: x.hour)
    df["Given min"] = df[dt_column].apply(lambda x: x.minute)
    
    df["Week Day"] = df[dt_column].apply(lambda x: x.dayofweek)
    df["Month"] = df[dt_column].apply(lambda x: x.month)
    df["Quarter"] = df[dt_column].apply(lambda x: x.quarter)
    df.drop(dt_column, axis=1, inplace=True)
    
    cyclical = CyclicalFeatures(variables=cyclic_feats, drop_original=True)
    df = cyclical.fit_transform(df)
    
    return df

In [6]:
bf_df = make_time_cyclical(bf_df)
ln_df = make_time_cyclical(ln_df)
em_df = make_time_cyclical(em_df)
bt_df = make_time_cyclical(bt_df)
ot_df = make_time_cyclical(ot_df)

In [7]:
timecode_dict = {"BF":[bf_df, bf_intervals], "LN":[ln_df, ln_intervals],
                 "EM":[em_df, em_intervals], "BT":[bt_df, bt_intervals], "OT":[ot_df, ot_intervals]}

In [8]:
from ipywidgets import interact, Dropdown

su_widget = Dropdown(options = unique_su, description='SU Number')
med_widget = Dropdown()
# timecode_widget = Dropdown(description="Medication")

def update_med_options(*args):
    med_widget.options = su_med_data[su_med_data["su_number"]==su_widget.value]["Medication Type"].unique().tolist()
# def update_time_code_options(*args):
#     timecode_widget.options = su_med_data[(su_med_data["su_number"]==su_widget.value) & \
#                                           (su_med_data["Medication Type"]==med_widget.value)] \
#                                           ["Medication Time Code"].unique().tolist()
med_widget.observe(update_med_options) 
# timecode_widget.observe(update_time_code_options)

@interact(su_number=su_widget, Medication=med_widget)
def print_vals(su_number, Medication):
    print(f"Selected Service User: {su_number}")
    print(f"Selected Medication: {Medication}")

interactive(children=(Dropdown(description='SU Number', options=(1, 3, 6, 8, 10, 13, 14, 15, 17, 18, 22, 23, 2…

In [9]:
timecode_widget = widgets.Dropdown(
    options=["BF", "LN", "EM", "BT", "OT"],
    description='Time Code',
    disabled=False,
)
display(timecode_widget)

Dropdown(description='Time Code', options=('BF', 'LN', 'EM', 'BT', 'OT'), value='BF')

In [10]:
dose_widget = widgets.FloatText(
    value=1,
    description='Dose:',
    disabled=False
)
display(dose_widget)

FloatText(value=1.0, description='Dose:')

In [11]:
su_meds = su_med_data[(su_med_data["su_number"] == su_widget.value) & (su_med_data["Medication Type"] == med_widget.value) \
           & (su_med_data["Medication Time Code"] == timecode_widget.value)]

In [12]:
su_meds = su_meds.loc[su_meds.index.repeat(len(timecode_dict[timecode_widget.value][0]))].reset_index(drop=True)
time_df = timecode_dict[timecode_widget.value][0]
data_df = pd.concat([su_meds, time_df], axis=1)
data_df["Schedule Dose"] = dose_widget.value
data_df.drop("su_number", axis=1, inplace=True)

In [13]:
data_df["Medication Type"] = data_df["Medication Type"].astype('category')
data_df["Medication Time Code"] = data_df["Medication Time Code"].astype('category')

data_df["Route"] = data_df["Route"].astype('category')
data_df["Form"] = data_df["Form"].astype('category')

In [14]:
def predict_time(b):
    preds = trained_model.predict_proba(data_df)
    predict_time.data = str(timecode_dict[timecode_widget.value][1][preds[:,0].argmax()])
    with output_box:
        print(f"Recommended Time to administer this medicine is: {predict_time.data}")

In [15]:
predict_button = widgets.Button(
    description='Predict Time',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run button function',
    icon='play'
)
output_box = widgets.Output()
display(predict_button, output_box)
predict_button.on_click(predict_time)

Button(button_style='info', description='Predict Time', icon='play', style=ButtonStyle(), tooltip='Run button …

Output()